# Feature selection.

We use feature selection after wrangling for two reasons.

1. Obtain a set of good features that represents the current dataset.
2. Obtain the set of *not good* features that should be refined in the next wrangling step.

This happens in three steps.

1. A first preselection step removes obviously bad features.
2. A second preselection step removes features that have the same predictive capabilities, in order to prevent the final feature selection step to select.
3. A real feature selection step to make the final decision.

The following methods are implemented in this notebook.

1. A (baseline) random sampling based approach — done.
2. CHCGA — a genetic algorithm based approach — done.
3. SFFS — a forward selection based approach — done.
4. AdaBoost with decision stump approach — TODO.

Both (1) and (2) allow use to set a max running time.

In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from typing import Optional, List, Tuple, Callable
from tqdm.notebook import tqdm
from avatar.language import WranglingLanguage
from avatar.analysis import *

Load dataset.

In [2]:
titanic = pd.read_csv("../data/raw/demo/titanic.csv")
titanic.Survived = titanic.Survived.astype("category")
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Find transformed columns. Don't use replacement.

In [3]:
language = WranglingLanguage()
expanded = language.expand(titanic, target="Survived")
expanded

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,OneHot()(Parch)_5,OneHot()(Parch)_6,OneHot()(Embarked)_C,OneHot()(Embarked)_Q,OneHot()(Embarked)_S,"NaN(Pernot, Mr. Rene)(Name)_Name","NaN(Somerton, Mr. Francis William)(Name)_Name",WordToNumber()(Ticket)_Ticket,ModeImputation()(Cabin)_Cabin,ModeImputation()(Embarked)_Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,0,0,0,1,"Braund, Mr. Owen Harris","Braund, Mr. Owen Harris",NaN,B96 B98,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,0,1,0,0,"Cumings, Mrs. John Bradley (Florence Briggs Th...","Cumings, Mrs. John Bradley (Florence Briggs Th...",NaN,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,0,0,0,1,"Heikkinen, Miss. Laina","Heikkinen, Miss. Laina",NaN,B96 B98,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,0,0,0,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)","Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803.0,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,0,0,0,1,"Allen, Mr. William Henry","Allen, Mr. William Henry",373450.0,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,...,0,0,0,0,1,"Montvila, Rev. Juozas","Montvila, Rev. Juozas",211536.0,B96 B98,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,...,0,0,0,0,1,"Graham, Miss. Margaret Edith","Graham, Miss. Margaret Edith",112053.0,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,...,0,0,0,0,1,"Johnston, Miss. Catherine Helen ""Carrie""","Johnston, Miss. Catherine Helen ""Carrie""",NaN,B96 B98,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,...,0,0,1,0,0,"Behr, Mr. Karl Howell","Behr, Mr. Karl Howell",111369.0,C148,C


## Pruning

Remove some features that are not appropriate and don't need more wrangling.

In [4]:
from avatar.selection import *


pruner = StackedFilter([ConstantFilter(),
                        IdenticalFilter()])
pruned = pruner.select(expanded)
pruned

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,OneHot()(Parch)_5,OneHot()(Parch)_6,OneHot()(Embarked)_C,OneHot()(Embarked)_Q,OneHot()(Embarked)_S,"NaN(Pernot, Mr. Rene)(Name)_Name","NaN(Somerton, Mr. Francis William)(Name)_Name",WordToNumber()(Ticket)_Ticket,ModeImputation()(Cabin)_Cabin,ModeImputation()(Embarked)_Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,0,0,0,1,"Braund, Mr. Owen Harris","Braund, Mr. Owen Harris",NaN,B96 B98,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,0,0,1,0,0,"Cumings, Mrs. John Bradley (Florence Briggs Th...","Cumings, Mrs. John Bradley (Florence Briggs Th...",NaN,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,0,0,0,1,"Heikkinen, Miss. Laina","Heikkinen, Miss. Laina",NaN,B96 B98,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,0,0,0,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)","Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803.0,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,0,0,0,1,"Allen, Mr. William Henry","Allen, Mr. William Henry",373450.0,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,...,0,0,0,0,1,"Montvila, Rev. Juozas","Montvila, Rev. Juozas",211536.0,B96 B98,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,...,0,0,0,0,1,"Graham, Miss. Margaret Edith","Graham, Miss. Margaret Edith",112053.0,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,...,0,0,0,0,1,"Johnston, Miss. Catherine Helen ""Carrie""","Johnston, Miss. Catherine Helen ""Carrie""",NaN,B96 B98,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,...,0,0,1,0,0,"Behr, Mr. Karl Howell","Behr, Mr. Karl Howell",111369.0,C148,C


## Preselection

Preselect features that will never be appropriate. These can still be wrangled.

* Remove columns with too many missing values.
* Columns consisting of unique, categorical features are removed.

In [5]:
from avatar.selection import *
    
preselector = StackedFilter([BijectiveFilter(),
                             UniqueFilter(),
                             MissingFilter()])
preselected = preselector.select(expanded)
preselected

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,...,OneHot()(Parch)_3,OneHot()(Parch)_4,OneHot()(Parch)_5,OneHot()(Parch)_6,OneHot()(Embarked)_C,OneHot()(Embarked)_Q,OneHot()(Embarked)_S,WordToNumber()(Ticket)_Ticket,ModeImputation()(Cabin)_Cabin,ModeImputation()(Embarked)_Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,S,...,0,0,0,0,0,0,1,NaN,B96 B98,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C,...,0,0,0,0,1,0,0,NaN,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,...,0,0,0,0,0,0,1,NaN,B96 B98,S
3,4,1,1,female,35.0,1,0,113803,53.1000,S,...,0,0,0,0,0,0,1,113803.0,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,S,...,0,0,0,0,0,0,1,373450.0,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,S,...,0,0,0,0,0,0,1,211536.0,B96 B98,S
887,888,1,1,female,19.0,0,0,112053,30.0000,S,...,0,0,0,0,0,0,1,112053.0,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,S,...,0,0,0,0,0,0,1,NaN,B96 B98,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C,...,0,0,0,0,1,0,0,111369.0,C148,C


We sample a subset of the data with at least one row containing no NaNs.

In [6]:
sampler = WeightedColumnSampler(preselected)
sampled = sampler.sample()
sampled

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Split(-)(Name)_0,"Split(,)(Name)_0",...,OneHot()(Embarked)_Q,OneHot()(Embarked)_S,ModeImputation()(Cabin)_Cabin,ModeImputation()(Embarked)_Embarked,Age,Embarked,Split( )(Name)_3,ExtractNumber()(Ticket)_0,"ExtractWord([Rev, Mrs, Mr, Dr, Miss, Master])(Name)_0",WordToNumber()(Ticket)_Ticket
0,1,0,3,male,1,0,A/5 21171,7.2500,"Braund, Mr. Owen Harris",Braund,...,0,1,B96 B98,S,22.0,S,Harris,5.0,Mr,NaN
1,2,1,1,female,1,0,PC 17599,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Cumings,...,0,0,C85,C,38.0,C,Bradley,17599.0,Mrs,NaN
2,3,1,3,female,0,0,STON/O2. 3101282,7.9250,"Heikkinen, Miss. Laina",Heikkinen,...,0,1,B96 B98,S,26.0,S,None,2.0,Miss,NaN
3,4,1,1,female,1,0,113803,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Futrelle,...,0,1,C123,S,35.0,S,Heath,113803.0,Mrs,113803.0
4,5,0,3,male,0,0,373450,8.0500,"Allen, Mr. William Henry",Allen,...,0,1,B96 B98,S,35.0,S,Henry,373450.0,Mr,373450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,0,0,211536,13.0000,"Montvila, Rev. Juozas",Montvila,...,0,1,B96 B98,S,27.0,S,None,211536.0,Rev,211536.0
887,888,1,1,female,0,0,112053,30.0000,"Graham, Miss. Margaret Edith",Graham,...,0,1,B42,S,19.0,S,Edith,112053.0,Miss,112053.0
888,889,0,3,female,1,2,W./C. 6607,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""",Johnston,...,0,1,B96 B98,S,NaN,S,Helen,6607.0,Miss,NaN
889,890,1,1,male,0,0,111369,30.0000,"Behr, Mr. Karl Howell",Behr,...,0,0,C148,C,26.0,C,Howell,111369.0,Mr,111369.0


Next, we look for features with the same predictive power using a wrapper approach. A decision stump is learned for each feature individually and the predictions for this stump are compared. Features that make the same predictions are pruned.

In [7]:
from avatar.selection import IterativeFilter


preselected = IterativeFilter().select(sampled, target="Survived")
preselected

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Split(-)(Name)_0,Split(.)(Name)_0,...,OneHot()(Embarked)_Q,OneHot()(Embarked)_S,ModeImputation()(Cabin)_Cabin,ModeImputation()(Embarked)_Embarked,Age,Embarked,Split( )(Name)_3,ExtractNumber()(Ticket)_0,"ExtractWord([Rev, Mrs, Mr, Dr, Miss, Master])(Name)_0",WordToNumber()(Ticket)_Ticket
0,1,0,3,male,1,0,A/5 21171,7.2500,"Braund, Mr. Owen Harris","Braund, Mr",...,0,1,B96 B98,S,22.0,S,Harris,5.0,Mr,NaN
1,2,1,1,female,1,0,PC 17599,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...","Cumings, Mrs",...,0,0,C85,C,38.0,C,Bradley,17599.0,Mrs,NaN
2,3,1,3,female,0,0,STON/O2. 3101282,7.9250,"Heikkinen, Miss. Laina","Heikkinen, Miss",...,0,1,B96 B98,S,26.0,S,None,2.0,Miss,NaN
3,4,1,1,female,1,0,113803,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)","Futrelle, Mrs",...,0,1,C123,S,35.0,S,Heath,113803.0,Mrs,113803.0
4,5,0,3,male,0,0,373450,8.0500,"Allen, Mr. William Henry","Allen, Mr",...,0,1,B96 B98,S,35.0,S,Henry,373450.0,Mr,373450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,0,0,211536,13.0000,"Montvila, Rev. Juozas","Montvila, Rev",...,0,1,B96 B98,S,27.0,S,None,211536.0,Rev,211536.0
887,888,1,1,female,0,0,112053,30.0000,"Graham, Miss. Margaret Edith","Graham, Miss",...,0,1,B42,S,19.0,S,Edith,112053.0,Miss,112053.0
888,889,0,3,female,1,2,W./C. 6607,23.4500,"Johnston, Miss. Catherine Helen ""Carrie""","Johnston, Miss",...,0,1,B96 B98,S,NaN,S,Helen,6607.0,Miss,NaN
889,890,1,1,male,0,0,111369,30.0000,"Behr, Mr. Karl Howell","Behr, Mr",...,0,0,C148,C,26.0,C,Howell,111369.0,Mr,111369.0


### Feature Selector: Evaluator

Wrapping evaluation in a class saves the time of converting data for MERCS and allows us to reuse the same split in every iteration.

In [8]:
from avatar.analysis import FeatureEvaluator

mask = np.random.randint(2, size=len(preselected.columns))
mask = np.zeros_like(mask)

evaluator_shap = FeatureEvaluator(folds=10, method="shap")
evaluator_shap.fit(preselected, target="Survived")


evaluator_fimp = FeatureEvaluator(folds=10)
evaluator_fimp.fit(preselected, target="Survived")

In [9]:
df = pd.DataFrame()
df['cols']=preselected.columns
df['shap']=evaluator_shap.importances(mask)
df['fimp']=evaluator_fimp.importances(mask)

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


In [10]:
df.sort_values(by='fimp', ascending=False).head(10)

,cols,shap,fimp
3,Sex,0.293688,0.269746
14,Split(. )(Ticket)_0,0.120592,0.142029
40,Age,0.072762,0.082374
7,Fare,0.086405,0.071322
42,Split( )(Name)_3,0.026447,0.045464
13,Split( )(Name)_2,0.033299,0.040023
0,PassengerId,0.038740,0.035895
10,Split(.)(Name)_1,0.023885,0.033280
45,WordToNumber()(Ticket)_Ticket,0.016640,0.032531
12,Split( )(Name)_0,0.007894,0.027534


In [11]:
df.sort_values(by='shap', ascending=False).head(10)

,cols,shap,fimp
3,Sex,0.293688,0.269746
14,Split(. )(Ticket)_0,0.120592,0.142029
7,Fare,0.086405,0.071322
40,Age,0.072762,0.082374
43,ExtractNumber()(Ticket)_0,0.040551,0.024294
15,Split(.)(Ticket)_0,0.040246,0.010194
0,PassengerId,0.038740,0.035895
8,Split(-)(Name)_0,0.037188,0.018841
13,Split( )(Name)_2,0.033299,0.040023
17,Split( )(Ticket)_0,0.032869,0.013151


## Feature selection


Next, we can take a look at actual feature selection. Three wrapper methods are implemented.

* Randomly sampling columns, training a model and getting the feature relevances.
* A genetic approach, which is similar but should combine features slightly better. We perform a small experiment on whether to fix the genome size.
* Classic sequential and backwards sequential feature selection.

The idea is similar; the feature selection algorithms return sets of feature importances and the associated scores.

### Mask-Generation: Random

Randomly sample subsets of features, evaluate and get feature relevances.

In [12]:
from avatar.selection import SamplingSelector


ss = SamplingSelector(iterations=100, evaluator=evaluator_shap)
ss.fit(preselected, target="Survived")

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

In [13]:
ss.scores()

array([3.70911225e-02, 0.00000000e+00, 4.63793947e-02, 2.06420976e-01,
       1.74816265e-02, 6.65046328e-03, 2.97172379e-02, 5.01485543e-02,
       2.32443540e-02, 2.04494694e-02, 2.05154953e-02, 3.26401749e-02,
       2.54609150e-02, 2.28966481e-02, 3.13412167e-02, 3.03426195e-02,
       2.22444133e-02, 2.78827035e-02, 2.69322176e-02, 4.67454718e-03,
       3.39050834e-02, 3.41210981e-03, 5.66463898e-03, 4.47006746e-04,
       5.62282073e-04, 9.29364839e-04, 1.36599305e-04, 5.52786067e-05,
       7.48821247e-03, 3.48775618e-03, 2.28711884e-03, 1.50166456e-05,
       7.50141004e-05, 8.34699380e-04, 6.13514757e-05, 2.42675353e-03,
       2.90995973e-03, 6.14602496e-03, 1.50814307e-02, 6.94613832e-03,
       3.88761995e-02, 7.56889545e-03, 2.34832036e-02, 2.83743935e-02,
       9.77275580e-02, 2.85837604e-02])

In [14]:
ss._fimps.shape

(100, 46)

In [15]:
ss.ordered()

Index(['Sex', 'ExtractWord([Rev, Mrs, Mr, Dr, Miss, Master])(Name)_0', 'Fare',
       'Pclass', 'Age', 'PassengerId', 'OneHot()(Pclass)_3',
       'Split(')(Name)_0', 'Split(. )(Ticket)_0', 'Split(.)(Ticket)_0',
       'Ticket', 'WordToNumber()(Ticket)_Ticket', 'ExtractNumber()(Ticket)_0',
       'Split( )(Ticket)_0', 'Split(/)(Ticket)_0', 'Split( )(Name)_0',
       'Split( )(Name)_3', 'Split(-)(Name)_0', 'Split( )(Name)_2',
       'Split(./)(Ticket)_0', 'Split(.)(Name)_1', 'Split(.)(Name)_0', 'SibSp',
       'ModeImputation()(Cabin)_Cabin', 'Embarked', 'OneHot()(Parch)_0',
       'ModeImputation()(Embarked)_Embarked', 'Parch', 'OneHot()(Embarked)_S',
       'OneHot()(SibSp)_1', 'OneHot()(Pclass)_2', 'OneHot()(Parch)_1',
       'OneHot()(SibSp)_0', 'OneHot()(Embarked)_Q', 'OneHot()(Embarked)_C',
       'OneHot()(Parch)_2', 'OneHot()(SibSp)_4', 'OneHot()(Parch)_5',
       'OneHot()(SibSp)_3', 'OneHot()(SibSp)_2', 'OneHot()(SibSp)_5',
       'OneHot()(Parch)_4', 'OneHot()(Parch)_6', 'One

### Mask Generation: Genetic

The CHC Genetic Algorithm for feature selection. Uses

* Cross-generational elitist selection
* Heterogeneous recombination
* and Cataclysmic mutation

for maintaining diversity and avoiding stagnation.

After the final population is obtained, combine importances from this population.

In [16]:
from avatar.selection import CHCGASelector, Population, Individual
    

gas = CHCGASelector(iterations=100, evaluator=evaluator_shap)
gas.fit(preselected, target="Survived")

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

In [17]:
gas.scores()

array([0.00000000e+00, 0.00000000e+00, 3.87455512e-02, 2.66707397e-01,
       2.71058115e-02, 8.20543975e-03, 5.33495575e-02, 7.87231185e-02,
       3.34383657e-02, 2.67193524e-02, 0.00000000e+00, 3.17336776e-02,
       3.96765939e-02, 0.00000000e+00, 9.40260519e-03, 5.52558590e-03,
       5.20812628e-02, 1.46112053e-02, 2.55122455e-02, 2.35401634e-03,
       6.78579593e-02, 8.58532948e-04, 5.19535950e-03, 2.50855082e-04,
       5.73704521e-05, 0.00000000e+00, 8.14832249e-05, 0.00000000e+00,
       2.52456411e-03, 1.62790743e-03, 2.73117235e-04, 0.00000000e+00,
       0.00000000e+00, 8.58768162e-04, 0.00000000e+00, 2.84482899e-03,
       1.68110921e-03, 1.19821801e-02, 1.16014395e-02, 5.83924635e-03,
       0.00000000e+00, 5.98625507e-03, 0.00000000e+00, 7.81783329e-02,
       8.84089048e-02, 0.00000000e+00])

In [18]:
gas.ordered()

Index(['Sex', 'ExtractWord([Rev, Mrs, Mr, Dr, Miss, Master])(Name)_0', 'Fare',
       'ExtractNumber()(Ticket)_0', 'OneHot()(Pclass)_3', 'Ticket',
       'Split(./)(Ticket)_0', 'Split( )(Name)_0', 'Pclass', 'Split(-)(Name)_0',
       'Split(')(Name)_0', 'SibSp', 'Split(.)(Name)_0', 'Split(/)(Ticket)_0',
       'Split( )(Ticket)_0', 'OneHot()(Embarked)_S',
       'ModeImputation()(Cabin)_Cabin', 'Split(. )(Ticket)_0', 'Parch',
       'Embarked', 'ModeImputation()(Embarked)_Embarked', 'Split(.)(Ticket)_0',
       'OneHot()(SibSp)_1', 'OneHot()(Embarked)_C', 'OneHot()(Parch)_0',
       'OneHot()(Pclass)_2', 'OneHot()(Embarked)_Q', 'OneHot()(Parch)_1',
       'OneHot()(Parch)_5', 'OneHot()(SibSp)_0', 'OneHot()(Parch)_2',
       'OneHot()(SibSp)_2', 'OneHot()(SibSp)_5', 'OneHot()(SibSp)_3',
       'Survived', 'WordToNumber()(Ticket)_Ticket', 'Split(.)(Name)_1',
       'Split( )(Name)_2', 'OneHot()(SibSp)_4', 'OneHot()(SibSp)_8',
       'OneHot()(Parch)_3', 'OneHot()(Parch)_4', 'OneHot()(Par

### Mask Generation: SFFS

Sequential Forward Floating Selection. We don't use the adaptive version as there will often be many columns and that is too slow.

In [20]:
from avatar.selection import SFFSelector
    

sffs = SFFSelector(iterations=40, evaluator=evaluator_shap)
sffs.fit(preselected, target="Survived")

[-1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

Adding Sex


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

Adding Fare


...

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

Adding Split(.)(Name)_0


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

Adding OneHot()(Embarked)_S


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.

In [21]:
sffs.scores()

array([0.00000000e+00, 0.00000000e+00, 1.75294068e-01, 3.75353191e-01,
       5.94550568e-03, 9.17428289e-03, 0.00000000e+00, 1.78969829e-01,
       0.00000000e+00, 7.15992132e-03, 0.00000000e+00, 6.72410704e-03,
       2.36252863e-02, 8.37494655e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.91091938e-03,
       0.00000000e+00, 0.00000000e+00, 1.12270947e-02, 1.09287582e-03,
       0.00000000e+00, 2.73921198e-03, 4.70039179e-04, 1.57569075e-04,
       2.44549371e-03, 2.11231828e-03, 2.44057701e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.01558372e-04, 1.38693817e-02, 9.24172405e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       8.61955792e-02, 0.00000000e+00])

In [22]:
sffs.ordered()

Index(['Sex', 'Fare', 'Pclass',
       'ExtractWord([Rev, Mrs, Mr, Dr, Miss, Master])(Name)_0',
       'Split( )(Name)_2', 'Split( )(Name)_0', 'ModeImputation()(Cabin)_Cabin',
       'OneHot()(SibSp)_1', 'ModeImputation()(Embarked)_Embarked', 'Parch',
       'Split(.)(Name)_0', 'Split(')(Name)_0', 'SibSp', 'OneHot()(SibSp)_4',
       'OneHot()(Parch)_0', 'OneHot()(Parch)_2', 'OneHot()(Parch)_1',
       'OneHot()(Pclass)_2', 'OneHot()(SibSp)_2', 'OneHot()(SibSp)_5',
       'OneHot()(SibSp)_8', 'OneHot()(Embarked)_S', 'Split(. )(Ticket)_0',
       'Split(.)(Name)_1', 'Split(./)(Ticket)_0', 'Split(-)(Name)_0', 'Ticket',
       'Survived', 'Split(.)(Ticket)_0', 'WordToNumber()(Ticket)_Ticket',
       'Split( )(Ticket)_0', 'Split(/)(Ticket)_0', 'OneHot()(Pclass)_3',
       'OneHot()(SibSp)_0', 'OneHot()(SibSp)_3', 'OneHot()(Parch)_3',
       'OneHot()(Parch)_4', 'OneHot()(Parch)_5', 'OneHot()(Parch)_6',
       'OneHot()(Embarked)_C', 'OneHot()(Embarked)_Q', 'Age', 'Embarked',
       'Split(